## Ingest images

In [1]:
from buck.analysis.basics import ingest_images

fpath = "..\\images\\squared\\*.png"
images,ages = ingest_images(fpath)

## Split datasets

In [2]:
from buck.analysis.basics import split_data

Xtr_og, ytr_og, Xval, yval, Xte, yte_onehot, ages, l_map = split_data(images, ages)

## Homogenize data across classes

In [3]:
from buck.analysis.basics import homogenize_data

augment_multiplier = 25
X_train_pca, y_train_flat, X_test_pca, y_true, label_mapping, num_classes = homogenize_data(Xtr_og, ytr_og, Xte,yte_onehot, l_map, augment_multiplier)

  Class 0 (Age 1.5): 38 → 1725 samples
  Class 1 (Age 2.5): 47 → 1725 samples
  Class 2 (Age 3.5): 68 → 1725 samples
  Class 3 (Age 4.5): 50 → 1725 samples
  Class 4 (Age 5.5): 69 → 1725 samples


## Comparison classifier models

In [4]:
#from buck.analysis.optimize_model import compare_models
#
#compare_models(X_train_pca, y_train_flat, X_test_pca, y_true, num_classes, label_mapping)

## Optimize top-performing classifier

In [5]:
from buck.classifiers.random_forest import optimize_random_forest

optimals, accuracy = optimize_random_forest(X_train_pca, y_train_flat, X_test_pca, y_true)
print(accuracy)

Optimizing random state...


KeyError: 'max_features'